### This script is used to creat sequences using data from NHTS 2009¶

In [1]:
#import useful libraries
from savReaderWriter import *
from pandas import *
import numpy as np
from collections import Counter
from dateutil.parser import parse
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#Read data files
trp = read_csv('C:/Users/jiz13007/Documents/Profile of Existing Travelers and Exploring Transforability for AV futures/NHTS Datasets/2017/trippub.csv')
print trp.shape

(923572, 102)


In [6]:
print trp[['HOUSEID','PERSONID','TDTRPNUM','STRTTIME','ENDTIME','TRVLCMIN']].head(10)

    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  ENDTIME  TRVLCMIN
0  30000007         1         1      1000     1015        15
1  30000007         1         2      1510     1530        20
2  30000007         2         1       700      900       120
3  30000007         2         2      1800     2030       150
4  30000007         3         1       845      900        15
5  30000007         3         2      1430     1445        15
6  30000008         1         1      1115     1130        15
7  30000008         1         2      2330     2340        10
8  30000012         1         1       550      605        15
9  30000012         1         2       700      715        15


In [5]:
# First is to check if all trips made by a person starts from 1
trp['CHECK_TRPNUM']=trp.groupby(['HOUSEID','PERSONID'])['TDTRPNUM'].transform('min')
print sum(trp['CHECK_TRPNUM']==1), sum(trp['CHECK_TRPNUM']!=1) # no issue was found, all person trip IDs start from 1

923572 0


### Create atttributes

In [ ]:
'''STRTTIME:Trip start Time
0000- 2359'''
'''ENDTIME: Trip end time'''
'''TRVLCMIN: Trip duration in minutes
-9: Not ascertain
0-1200'''

In [10]:
print trp['STRTTIME'].min(), trp['STRTTIME'].max()
print trp['ENDTIME'].min(), trp['ENDTIME'].max()
print trp['TRVLCMIN'].min(),trp['TRVLCMIN'].max()

0 2359
0 2359
-9 1200


In [ ]:
''' Survey starts from 4:00 am to 3:59 am of the next day'''

In [27]:
trp['STRTTIME_R']=[a if a>=400 else (a+2400) for a in trp['STRTTIME'].tolist()]
trp['ENDTIME_R']=[a if a>=400 else (a+2400) for a in trp['ENDTIME'].tolist()]
print trp['STRTTIME_R'].min(), trp['ENDTIME_R'].min()
print trp['STRTTIME_R'].max(), trp['ENDTIME_R'].max()

400 401
2759 2758


In [28]:
# Convert STRTTIME for calculating activity duration
trp['STRT_HR']=trp['STRTTIME_R']//100
trp['STRT_MIN']=trp['STRTTIME_R']%100
trp['END_HR']=trp['ENDTIME_R']//100
trp['END_MIN']=trp['ENDTIME_R']%100
trp['STRTTIME_CONVERT']=trp['STRT_HR']*60+trp['STRT_MIN']
trp['ENDTIME_CONVERT']=trp['END_HR']*60+trp['END_MIN']
print trp['STRT_HR'].min(), trp['STRT_HR'].max()
print trp['STRT_MIN'].min(), trp['STRT_MIN'].max()
print trp['END_HR'].min(), trp['END_HR'].max()
print trp['END_MIN'].min(), trp['END_MIN'].max()
print trp['STRTTIME_CONVERT'].min(), trp['STRTTIME_CONVERT'].max()
print trp['ENDTIME_CONVERT'].min(), trp['ENDTIME_CONVERT'].max()
print trp[['TDTRPNUM','STRTTIME','START_HR','START_MIN','STRTTIME_CONVERT','ENDTIME','END_HR','END_MIN','ENDTIME_CONVERT']].head()

4 27
0 59
4 27
0 59
240 1679
241 1678
   TDTRPNUM  STRTTIME  START_HR  START_MIN  STRTTIME_CONVERT  ENDTIME  END_HR  \
0         1      1000        10          0               600     1015      10   
1         2      1510        15         10               910     1530      15   
2         1       700         7          0               420      900       9   
3         2      1800        18          0              1080     2030      20   
4         1       845         8         45               525      900       9   

   END_MIN  ENDTIME_CONVERT  
0       15              615  
1       30              930  
2        0              540  
3       30             1230  
4        0              540  


In [32]:
# Check if TRVLCMIN matches with endtime-strtime
trp['CHECK_TRVL']=trp['ENDTIME_CONVERT']-trp['STRTTIME_CONVERT']
print trp['CHECK_TRVL'].min(), trp['CHECK_TRVL'].max()
print sum((trp['CHECK_TRVL']!=trp['TRVLCMIN'])&(trp['TRVLCMIN']!=-9))
print trp.loc[(trp['CHECK_TRVL']!=trp['TRVLCMIN']),('HOUSEID','PERSONID','TDTRPNUM')].head()
print sum(trp['CHECK_TRVL']<=0), sum(trp['TRVLCMIN']==-9)

# NEED TO REMOVE ALL PERSONS WITH NEGATIVE OR ZERO TRIP DURATION

-1230 1200
0
       HOUSEID  PERSONID  TDTRPNUM
196   30000288         1         3
293   30000402         2         2
294   30000402         2         3
947   30001286         1         1
2753  30003515         1         2
66 1297


In [ ]:
'''WHYTRP1S: Trip purpose summary
1- Home
10- Work
20- School
30- Medical
40- Shopping
50- Social
70- Transport someone
80- Meals
97- Something elase'''

In [41]:
## mandatory
mandatory_filter=trp['WHYTRP1S'].isin([10,20])
print sum(mandatory_filter)
trp.loc[mandatory_filter, 'TYPE']=2

## maintenance
maintenance_filter=trp['WHYTRP1S'].isin([30,40,70,80])
print sum(maintenance_filter)
trp.loc[maintenance_filter, 'TYPE']=3

## discretionary
discretionary_filter=trp['WHYTRP1S'].isin([50,97])
print sum(discretionary_filter)
trp.loc[discretionary_filter, 'TYPE']=4

# home
home_filter=trp['WHYTRP1S']==1
print sum(home_filter)
trp.loc[home_filter, 'TYPE']=1

print trp['TYPE'].value_counts().sort_index()
print trp['TYPE'].isnull().sum()
print trp['WHYTRP1S'].value_counts().sort_index()

153987
329614
121194
318777
1.0    318777
2.0    153987
3.0    329614
4.0    121194
Name: TYPE, dtype: int64
0
1     318777
10    110590
20     43397
30     16784
40    184126
50    100284
70     56377
80     72327
97     20910
Name: WHYTRP1S, dtype: int64


In [ ]:
'''TRPTRANS: Transportation mode used on trip
-9- Not ascertained
-8- I don't know
-7- I prefer not to answer
1- walk
2- bicycle
3- Car
4- SUV
5- Van
6- Pickup truck
7- Golf cart/segway
8- motorcycle
9- RV
10- school bus
11- public or commuter bus
12- paratransit 
13- private/charter/tour/shuttle bus
14- city to city bus
15- armtrack
16- subway/elevated light rail
17-Taxi
18- Rental car
19- Airplane
20- Boat
97- Something else
'''

In [43]:
#auto
auto_filter=trp['TRPTRANS'].isin([3,4,5,6,18])
print sum(auto_filter)
trp.loc[auto_filter,'MODE']=1
#public transit
public_filter=trp['TRPTRANS'].isin([11,14,15,16])
print sum(public_filter)
trp.loc[public_filter,'MODE']=2
#non-motorized
nonmotor_filter=trp['TRPTRANS'].isin([1,2])
print sum(nonmotor_filter)
trp.loc[nonmotor_filter,'MODE']=3
#other mode
other_filter=trp['TRPTRANS'].isin([7,8,9,10,12,13,17,19,20,97])
print sum(other_filter)
trp.loc[other_filter,'MODE']=4
print trp['MODE'].value_counts().sort_index()
print trp['MODE'].isnull().sum(),'invalid mode records need to be removed'
print trp['TRPTRANS'].value_counts().sort_index()

797169
11210
89322
25855
1.0    797169
2.0     11210
3.0     89322
4.0     25855
Name: MODE, dtype: int64
16 invalid mode records need to be removed
-9          1
-8         13
-7          2
 1      81288
 2       8034
 3     396931
 4     229466
 5      60463
 6     108303
 7        826
 8       2088
 9        814
 10     11313
 11      6616
 12       624
 13      1581
 14       120
 15      1148
 16      3326
 17      2813
 18      2006
 19      1823
 20       458
 97      3515
Name: TRPTRANS, dtype: int64


In [51]:
## create filter to inidcate invalid records
purfilter=trp['TYPE'].isnull()
print sum(purfilter), 'invalid purpose filter'
modefilter=trp['MODE'].isnull()
print sum(modefilter), 'invalid mode filter'
trpminfilter=(trp['CHECK_TRVL']<=0)
print sum(trpminfilter), 'invalid duration'

trp.loc[(purfilter|modefilter|trpminfilter),'invalid_indicator']=1
trp['invalid_indicator'].replace(np.nan, 0, inplace=True)
print sum(trp['invalid_indicator']==1),'invalid trips'

## mark all trips made by persons who had one or more invalid record
trp['invalid_per']=trp.groupby(['HOUSEID','PERSONID'])['invalid_indicator'].transform('max')
print sum(trp['invalid_per']==1), 'all trips need to be removed'

0 invalid purpose filter
16 invalid mode filter
66 invalid duration
82 invalid trips
464 all trips need to be removed


In [53]:
# remove invalid trips and create new dataframe
trp2=trp.loc[trp['invalid_per']==0,:]
print trp2.shape, trp.shape, len(trp)-len(trp2)
print trp2['STRTTIME_CONVERT'].max(), 'latest start time'
print trp2['ENDTIME_CONVERT'].max(),'latest arrivial time'
print trp2['CHECK_TRVL'].min(),'shortest trip duration'
print trp2['CHECK_TRVL'].max(), 'longest trip duration'
print trp2['TYPE'].isnull().sum(), 'nan activity types'
print trp2['MODE'].isnull().sum(), 'nan mode types'
print trp2['STRTTIME_CONVERT'].isnull().sum(), 'nan start time'
print trp2['ENDTIME_CONVERT'].isnull().sum(), 'nan end time'
print trp2['CHECK_TRVL'].isnull().sum(), 'nan travel time'

 (923108, 118) (923572, 118) 464
1674 latest start time
1678 latest arrivial time
1 shortest trip duration
1200 longest trip duration
0 nan activity types
0 nan mode types
0 nan start time
0 nan end time
0 nan travel time


## Recode activity dwell time

In [54]:
trp2.sort_values(['HOUSEID','PERSONID','TDTRPNUM'], ascending=[True,True,True], inplace=True)

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [60]:
trp2['DWELTIME2']=trp2['DWELTIME']
trp2['DWELTIME2']=[x if x!=-9 else (1680-y) for x,y in zip(trp2['DWELTIME'].tolist(),trp2['ENDTIME_CONVERT'].tolist())]
trp2['FIRST_STRT']=[(x-240) if y==1 else 0 for x, y in zip(trp2['STRTTIME_CONVERT'].tolist(), trp2['TDTRPNUM'].tolist())]

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [64]:
print trp2[['HOUSEID','PERSONID','TDTRPNUM','STRTTIME','STRTTIME_CONVERT','ENDTIME','ENDTIME_CONVERT','TRVLCMIN','DWELTIME','DWELTIME2','FIRST_STRT']].head(10)

    HOUSEID  PERSONID  TDTRPNUM  STRTTIME  STRTTIME_CONVERT  ENDTIME  \
0  30000007         1         1      1000               600     1015   
1  30000007         1         2      1510               910     1530   
2  30000007         2         1       700               420      900   
3  30000007         2         2      1800              1080     2030   
4  30000007         3         1       845               525      900   
5  30000007         3         2      1430               870     1445   
6  30000008         1         1      1115               675     1130   
7  30000008         1         2      2330              1410     2340   
8  30000012         1         1       550               350      605   
9  30000012         1         2       700               420      715   

   ENDTIME_CONVERT  TRVLCMIN  DWELTIME  DWELTIME2  FIRST_STRT  
0              615        15       295        295         360  
1              930        20        -9        750           0  
2              

In [66]:
### check if the trip equals to 1440
trp2['SUM_DWEL']=trp2.groupby(['HOUSEID','PERSONID'])['DWELTIME2'].transform('sum')
trp2['SUM_TRVL']=trp2.groupby(['HOUSEID','PERSONID'])['CHECK_TRVL'].transform('sum')
trp2['SUM_FIR']=trp2.groupby(['HOUSEID','PERSONID'])['FIRST_STRT'].transform('first')
trp2['CHECK_DUR']=trp2['SUM_DWEL']+trp2['SUM_TRVL']+trp2['SUM_FIR']
print trp2['CHECK_DUR'].value_counts()
## remove the persons with invalid duration
trp3=trp2.loc[trp2['CHECK_DUR']==1440]
print trp3.shape

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


1440    923080
2700        11
2400         9
2669         5
2670         3
Name: CHECK_DUR, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [72]:
## remove the persons with invalid duration 
trp2['ZERO_DUR']=trp2.groupby(['HOUSEID','PERSONID'])['DWELTIME2'].transform('min')
print sum(trp2['ZERO_DUR']==0)
trp3=trp2.loc[(trp2['CHECK_DUR']==1440)|(trp2['ZERO_DUR']==0)]
print trp3.shape

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


20
(923080, 125)


In [73]:
# Recode activity type so that mode and actvity code can be difficiated
trp3.loc[trp3['MODE']==1,'MODE_RECODE']='A'
trp3.loc[trp3['MODE']==2,'MODE_RECODE']='B'
trp3.loc[trp3['MODE']==3,'MODE_RECODE']='C'
trp3.loc[trp3['MODE']==4,'MODE_RECODE']='D'
trp3.loc[trp3['TYPE']==1,'TYPE_RECODE']='E'
trp3.loc[trp3['TYPE']==2,'TYPE_RECODE']='F'
trp3.loc[trp3['TYPE']==3,'TYPE_RECODE']='G'
trp3.loc[trp3['TYPE']==4,'TYPE_RECODE']='H'
print trp3['TYPE_RECODE'].value_counts()
print trp3['TYPE'].value_counts()
print trp3['MODE_RECODE'].value_counts()
print trp3['MODE'].value_counts()
print trp3['TYPE_RECODE'].isnull().sum(),trp3['TYPE'].isnull().sum()
print trp3['MODE_RECODE'].isnull().sum(),trp3['MODE'].isnull().sum()

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


G    329398
E    318645
F    153943
H    121094
Name: TYPE_RECODE, dtype: int64
3.0    329398
1.0    318645
2.0    153943
4.0    121094
Name: TYPE, dtype: int64
A    796801
C     89238
D     25836
B     11205
Name: MODE_RECODE, dtype: int64
1.0    796801
3.0     89238
4.0     25836
2.0     11205
Name: MODE, dtype: int64
0 0
0 0


In [74]:
## Next is to create sequence/time series data for each individual
trp3['TRIPSEQ']=[str(a)*b for a,b in zip (trp3['MODE_RECODE'], trp3['CHECK_TRVL'])]
trp3['ACTSEQ']=[str(a)*b for a,b in zip (trp3['TYPE_RECODE'], trp3['DWELTIME2'])]
trp3['FIRSEQ']=['T'*b for b in trp3['FIRST_STRT']]
print trp3[['TDTRPNUM','MODE_RECODE','CHECK_TRVL','DWELTIME2','TRIPSEQ','ACTSEQ','FIRST_STRT','FIRSEQ']].head()

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


   TDTRPNUM MODE_RECODE  CHECK_TRVL  DWELTIME2  \
0         1           A          15        295   
1         2           A          20        750   
2         1           A         120        540   
3         2           A         150        450   
4         1           A          15        330   

                                             TRIPSEQ  \
0                                    AAAAAAAAAAAAAAA   
1                               AAAAAAAAAAAAAAAAAAAA   
2  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
3  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
4                                    AAAAAAAAAAAAAAA   

                                              ACTSEQ  FIRST_STRT  \
0  FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...         360   
1  EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...           0   
2  EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...         180   
3  FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...           0   
4  FFFFFFFFFFFFFFFFFFF

In [76]:
trp3['TRPSEQ_ALL']=trp3['FIRSEQ'].map(str)+trp3['TRIPSEQ'].map(str)
x=trp3.loc[0,'FIRSEQ']
x1=''.join(x)
print len(x1)
y=trp3.loc[0,'TRIPSEQ']
y1=''.join(y)
print len(y1)
z=trp3.loc[0,'TRPSEQ_ALL']
z1=''.join(z)
print len(z1)
print trp3[['FIRST_STRT','TRVLCMIN','DWELTIME2']].head()

360
15
375
   FIRST_STRT  TRVLCMIN  DWELTIME2
0         360        15        295
1           0        20        750
2         180       120        540
3           0       150        450
4         285        15        330


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [77]:
trp3['ACTIVITY']=trp3['TRPSEQ_ALL'].map(str)+trp3['ACTSEQ'].map(str)
print trp3[['TDTRPNUM','TRPSEQ_ALL','ACTSEQ','ACTIVITY']].head()
perseq=trp3.groupby(['HOUSEID','PERSONID'])['ACTIVITY'].apply(list)
print perseq.head()

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


   TDTRPNUM                                         TRPSEQ_ALL  \
0         1  TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...   
1         2                               AAAAAAAAAAAAAAAAAAAA   
2         1  TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...   
3         2  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
4         1  TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...   

                                              ACTSEQ  \
0  FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...   
1  EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...   
2  EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...   
3  FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...   
4  FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...   

                                            ACTIVITY  
0  TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...  
1  AAAAAAAAAAAAAAAAAAAAEEEEEEEEEEEEEEEEEEEEEEEEEE...  
2  TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...  
3  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [78]:
perseq=DataFrame(perseq)
perseq.reset_index(inplace=True)
print perseq.head()
print perseq.shape

    HOUSEID  PERSONID                                           ACTIVITY
0  30000007         1  [TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
1  30000007         2  [TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
2  30000007         3  [TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
3  30000008         1  [TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
4  30000012         1  [TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
(219115, 3)


In [79]:
# check if all sequences have length of 1440
indexLS=perseq.index.tolist()
for i in indexLS:
    seq=perseq.loc[i,'ACTIVITY'] # list of sequence
    seq1=''.join(seq)
    cnt=len(seq1)
    perseq.loc[i,'TXTSEQUENCE']=seq1
    perseq.loc[i,'SEQ_LENGTH']=cnt

print perseq['SEQ_LENGTH'].value_counts()

1440.0    219115
Name: SEQ_LENGTH, dtype: int64


In [80]:
perseq.to_pickle('C:/Users/jiz13007/Documents/Pattern Recognition/NHTS sequence/2017_sequence.pkl')
trp3.to_pickle('C:/Users/jiz13007/Documents/Pattern Recognition/NHTS sequence/2017_trips.pkl')